In [1]:
import pandas as pd #引入panda
import os   #引入graphviz環境
os.environ["PATH"] += os.pathsep + 'C:/Program Files/Graphviz/bin' #依graphviz安裝位置修改
from graphviz import Digraph, render #使用graphviz的diagraph(有向圖)、render(dot轉png用)模組

#宣告全域變數，ALLCONCEPTMAP存讀者資料，GRAPHFEATURE存製圖相關方法
global ALLCONCEPTMAP, GRAPHFEATURE 

reader_concept_map_dataset = "Reader_Concept_Map_Data.json"

ALLCONCEPTMAP = pd.read_json(reader_concept_map_dataset, encoding="utf-8").T
all_reader_number = len(ALLCONCEPTMAP) #計算讀者數量

In [2]:
def about_nodes(reader, reader_number):

    save_all_nodes = [] #宣告陣列，之後用於存取資料元素(節點)

    concept_map_data_elements = ALLCONCEPTMAP["data_elements"][reader]  #讀取個別讀者所有資料元素
    number_of_elements = int(len(concept_map_data_elements))  #計算元素數量

    #將所有元素相關資料讀出並存取
    for each_graph_node in range(number_of_elements):

        which_graph = concept_map_data_elements[each_graph_node]["in_graph"]  #哪種圖

        GRAPHFEATURE.attr("graph", {"label": '{}\l'.format(reader_number)})  #給圖方法另外加上label屬性

        if which_graph[0] == 1: #若是尋找圖

            element_id = "atr0_" + str(each_graph_node)  #節點id(弧會用到)
            element_no = concept_map_data_elements[each_graph_node]["element_number"] #元素出現順序(論文中多未用到)
            element_value = concept_map_data_elements[each_graph_node]["element_content"]  #元素讀者值
            element_attribute = concept_map_data_elements[each_graph_node]["element_attribute"] #元素讀者屬性
            element_ontology_property = concept_map_data_elements[each_graph_node]["in_ontology"][0] #元素知識本體屬性
            element_ontology_entity = concept_map_data_elements[each_graph_node]["in_ontology"][1] #元素知識本體實體
            element_code = concept_map_data_elements[each_graph_node]["element_code"]  #元素研究定義種類(41)
            element_class = concept_map_data_elements[each_graph_node]["element_class"]  #元素研究定義類別(9)

            #將上述所有元素資料做成字典
            node_attribute_list = {"id": element_id,
                                   "no": element_no,
                                   "value": element_value,
                                   "data_elements": element_attribute,
                                   "ontology_property": element_ontology_property,
                                   "ontology_entity": element_ontology_entity, 
                                   "code": element_code,
                                   "class": element_class,
                                   "reader_number": reader_number}

            #儲存所有讀者元素字典資料
            save_all_nodes += [node_attribute_list]

    #回傳元素字典資料
    return save_all_nodes

In [3]:
def about_edges(reader, reader_number):

    save_all_edges = []

    concept_map_relations = ALLCONCEPTMAP["relationships"][reader]
    number_of_relations = len(concept_map_relations)

    for each_graph_relation in range(number_of_relations):

        realtions_of_diagraph = concept_map_relations[each_graph_relation]["relationship_attribute"]  #讀取個別讀者所有關係
        number_of_diagraph_relations = int(len(realtions_of_diagraph))  #subject元素數量  注意!!!雙向路徑的subject沒有拆成兩個單向

        #讀取個別讀者單一關係
        for each_relation in range(number_of_diagraph_relations):
                
            which_graph =  concept_map_relations[each_graph_relation]["relationship_attribute"][0]["in_graph"][0]

            if which_graph == 1:

                edge_subject_element = concept_map_relations[each_graph_relation]["subject_element"]  #起點元素
                edge_object_element = concept_map_relations[each_graph_relation]["relationship_attribute"][each_relation]["object_element"]  #終點元素
                edge_predicate = concept_map_relations[each_graph_relation]["relationship_attribute"][each_relation]["predicate"]  #關係內容
                edge_line = concept_map_relations[each_graph_relation]["relationship_attribute"][each_relation]["relationship_edge_line"]  #實線或虛線(關係強度實>虛)
                edge_direct = concept_map_relations[each_graph_relation]["relationship_attribute"][each_relation]["relationship_edge_direct"]  #方向性(單向、雙向、無向)

                #把上述關係相關資料做成字典
                edge_attribute_list = {"predicate": edge_predicate,
                                       "direct": edge_direct,
                                       "line": edge_line, 
                                       "subject element": edge_subject_element,
                                       "object element": edge_object_element,
                                       "reader_number": reader_number}

                #儲存各關係資料
                save_all_edges += [edge_attribute_list]

    #回傳關係資料
    return save_all_edges

In [4]:
def graph_maker(reader, reader_number):

    for list_the_nodes in range(len(node_has)):

        #呈現元素(節點)相關屬性
        GRAPHFEATURE.node(name = '{}'.format(node_has[list_the_nodes]["id"]),
                          label = '{}'.format(r'{}\l\n{}\n\n{}\r'.format(node_has[list_the_nodes]["no"],
                                                                         node_has[list_the_nodes]["value"],
                                                                         node_has[list_the_nodes]["data_elements"])))

    for list_the_edges in range(len(edge_has)):
        
        #呈現關係(路徑)相關屬性
        GRAPHFEATURE.edge(edge_has[list_the_edges]["subject element"],
                          edge_has[list_the_edges]["object element"],
                          label = edge_has[list_the_edges]["predicate"],
                          style = edge_has[list_the_edges]["line"],
                          dir = edge_has[list_the_edges]["direct"])

    #回傳各張圖
    return GRAPHFEATURE

In [5]:
def list_report(node_list, edge_list, node_has, edge_has, report_data):
    #元素與關係報表方法

    node_list += node_has  #存取所有讀者元素資料
    edge_list += edge_has  #存取所有讀者關係資料

    report_data = {"node": node_list, "edge": edge_list}  #做成字典並回傳

    return report_data

In [6]:
node_list = []  #儲存所有元素用
edge_list = []  #儲存所有關係用

report_data = []  #儲存元素、關係資料用
#edge_report = []

for reader in range(all_reader_number):

    reader_number = "RR_"+'{:0>2d}'.format(reader+1)

    GRAPHFEATURE = Digraph(node_attr = {"shape": "box",
                                         "height": "1", "width": "2",
                                         "format": "png",
                                         "fontname": "Microsoft JhengHei",
                                         "format": "png"},

                            edge_attr={"fontname": "Microsoft JhengHei"},

                            format = "png",

                            graph_attr={"labelloc": "t",
                                       "fontsize": "26",
                                       "labeljust": "left"})

    node_has = about_nodes(reader, reader_number)  #執行上方節點方法，存為變數node_has
    edge_has = about_edges(reader, reader_number)  #執行上方路徑方法，存為變數edge_has
    
    graph_maker(reader, reader_number)  #執行上方製圖方法

    GRAPHFEATURE#.render(reader_number, directory="C:\\Users\\jerry\\OneDrive\\桌面\\show")  #輸出圖片，檔名為讀者編碼，檔案輸出至指定資料夾

    report_all = list_report(node_list, edge_list, node_has, edge_has, report_data)  #執行上方元素與關係報表方法



In [8]:
#元素報表
node_report = pd.DataFrame(report_all["node"]).set_index("reader_number")

node_report#.to_csv("node report.csv", encoding = "utf-8")


,id,no,value,data_elements,ontology_property,ontology_entity,code,class
reader_number,,,,,,,,
RR_01,atr0_0,1,Hogwarts Express,書中特定交通工具,frbrer:P3049(f),Object(fictional),角色相關資訊,故事內容
RR_01,atr0_1,2,9 3/4,書中地名,frbrer:P3051(f),Place(fictional),虛構地點,故事內容
RR_01,atr0_2,3,Quidditch,書中一活動名,frbrer:P3050(f),Event(fictional),虛構事件或活動,故事內容
RR_01,atr0_3,na,J. K. Rowling,作者,frbrer:P3039,Person,主要作者,作者
RR_01,atr0_4,na,Harry Potter1,(1)書名(2)作品本身,frbrer:P3001; [frbrer:P3008],Work; [Expression],書名; 內容全文,作品; 故事內容
...,...,...,...,...,...,...,...,...
RR_33,atr0_2,3,一輛列車上,書的封面,frbrer:P3051(f); [Final_P18],Place(fictional); [Expression],封面插圖,外觀尺寸
RR_33,atr0_3,4,2015…風景,他人評語,frbrer:P3018,Expression,他人推薦或評論,評論
RR_33,atr0_4,5,知名作家…等候我,他人評語,frbrer:P3018,Expression,他人推薦或評論,評論


In [9]:
#關係報表
edge_list = pd.DataFrame(report_all["edge"])

for edge_number in edge_list.index:

    if edge_list.loc[edge_number, "line"] == "":

        edge_list.loc[edge_number, "line"] = "實線"
    
    else:

        edge_list.loc[edge_number, "line"] = "虛線"

    if edge_list.loc[edge_number, "direct"] == "":

        edge_list.loc[edge_number, "direct"] = ">"

    elif edge_list.loc[edge_number, "direct"] =="none":

        edge_list.loc[edge_number, "direct"] = "無向"

    else:

        edge_list.loc[edge_number, "direct"] = "雙向"

edge_report = pd.DataFrame(edge_list.set_index("reader_number"), columns = (["subject element", "direct", "line", "predicate", "object element"]))

edge_report#.to_csv("relationship report.csv", encoding = "utf-8")

,subject element,direct,line,predicate,object element
reader_number,,,,,
RR_01,atr0_3,>,實線,[認識]描述、想像,atr0_5
RR_01,atr0_3,>,實線,創作,atr0_4
RR_01,atr0_3,>,實線,描述 設定 想像,atr0_6
RR_01,atr0_4,無向,實線,角色,atr0_6
RR_01,atr0_4,>,實線,改變,atr0_3
...,...,...,...,...,...
RR_33,atr0_2,>,實線,發生地點,atr0_0
RR_33,atr0_3,>,實線,評論,atr0_0
RR_33,atr0_4,>,實線,評論,atr0_0
